In [1]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [3]:
def max_drawdown(X):
    mdd = 0
    peak = X[0]
    for x in X:
        if x > peak: 
            peak = x
        dd = (peak - x) / peak
        if dd > mdd:
            mdd = dd
    return mdd

In [4]:
cd 'C:\Users\dsi\Documents\Python Scripts\data'

C:\Users\dsi\Documents\Python Scripts\data


In [5]:
SP = pd.DataFrame.from_csv('SP Monthly.csv')

In [6]:
SP.columns = ['AdjClose','T Bill']
# SP['T Bill'] = TB.fillna(method='ffill')
# SP['T Bill'] = SP['T Bill']*0

In [7]:
FiveYrMax = pd.rolling_max(SP['AdjClose'],60)

In [8]:
SP['Pullback']=(SP['AdjClose']-FiveYrMax)/FiveYrMax

In [9]:
SP['T Bill'] = SP['T Bill']/100

In [10]:
SP['Date'] = SP.index

In [11]:
SP = SP[['Date','AdjClose','Pullback','T Bill']]

In [12]:
SP.index= list(range(len(SP.index)))

In [13]:
SP['T Bill'] = SP['T Bill']*0
SP = SP[0:775]

In [14]:
# d = dt.datetime(2007,1,1)
# SP = SP[SP['Date']<= d]

In [19]:
MTM = []
sharpe = []
mdd = []
entry = []

In [20]:
thres = [0.1,0.1,0.15,0.4]
alloc =[1,0,0,0]

for thre in [thre / 100.0 for thre in range(1,51)]:
    thres = [thre,1,0.3,0.4]
    InitialCash = 1000000
    StartDate = 60
    Time = 0
      # set allocation
    N,M = SP.shape
    p = np.zeros([N,1])
#     p1 = np.zeros([N,1])
#     p2 = np.zeros([N,1])
#     p3 = np.zeros([N,1])
#     p4 = np.zeros([N,1])
#     p5 = np.zeros([N,1])
#     p6 = np.zeros([N,1])
#     plist = [p1,p2,p3,p4,p5,p6]
    val = np.zeros([N,1])
    Cash = np.zeros([N,1])
    Cash[StartDate-1] = InitialCash
    val[StartDate-1] = InitialCash
    buySignal = [False,False,False,False,False,False]
    levels = ['1','2','3','4']
    position = {elem : [] for elem in levels}

    for x in range(StartDate,len(SP.index)): 

    # Compound free cash using 3-month T-bill rate (divided by three to make it monthly)

        Cash[x] = Cash[x-1]*(1+SP.loc[x,'T Bill'])
        p[x] = p[x-1]

    # Check selling signals for all positions at all levels

        for key in position.keys():
            if position[key]:
                for pos in range(len(position[key])):
                    if (x-position[key][pos][0])%12==0:
                        #sell
                        mult_helper = (x-position[key][pos][0])/12
                        mult = (5-mult_helper)/(6-mult_helper)
                        Cash[x] = Cash[x] + SP.loc[x,'AdjClose']*position[key][pos][1]*(1-mult)
                        p[x] = p[x] - position[key][pos][1]*(1-mult)
                        position[key][pos][1] = position[key][pos][1]*mult
                for pos in position[key]:       
                    if pos[1] == 0:
                        del position[key][position[key]==pos]

    # Check if position at certain level is zero, switch to buy if yes    

        for key in position.keys():
            n = int(key)-1
            if not position[key]:
                buySignal[n] = False

    # Check if new high occurs 

        if SP.loc[x,'Pullback'] == 0:
            for i in range(len(buySignal)):
                buySignal[i] = False

    # Check buying signal and make order

        for key in position.keys():
                n = int(key)-1          
                if(int(key) != len(levels)):
                    if (SP.loc[x-1,'Pullback']> -thres[n]) & (SP.loc[x,'Pullback'] <= -thres[n]) & (SP.loc[x,'Pullback'] > -thres[n+1]) & (not buySignal[n]):
                        #buy
                        share = Cash[x]*alloc[n]/SP.loc[x,'AdjClose']
                        position[key].append([x,share])
                        Cash[x] = Cash[x]*(1-alloc[n])
                        p[x] = p[x] + share
                        buySignal[n] = True
                else:
                    if (SP.loc[x-1,'Pullback']> -thres[n]) & (SP.loc[x,'Pullback'] <= -thres[n]) & (not buySignal[n]):
                        #buy
                        share = Cash[x]*alloc[n]/SP.loc[x,'AdjClose']
                        position[key].append([x,share])
                        Cash[x] = Cash[x]*(1-alloc[n])
                        p[x] = p[x] + share
                        buySignal[n] = True

    # Track total position held at different levels 

        for key in position.keys():
            if position[key]:
                for pos in range(len(position[key])):
                    n = int(key)-1
#                     plist[n][x] = plist[n][x] + position[key][pos][1]

        if p[x] < 0.0001: p[x] = 0.0                 
        val[x] = Cash[x] + p[x]*SP.loc[x,'AdjClose']


    #     SP['Cash'] = Cash
    #     SP['Position'] = p
    #     SP['P1'] = plist[0]
    #     SP['P2'] = plist[1] 
    #     SP['P3'] = plist[2]
    #     SP['P4'] = plist[3]
    #     SP['P5'] = plist[4]
    #     SP['P6'] = plist[5]
    SP['MTM'] = val
    #     SP = SP[['Date','AdjClose','Pullback','T Bill','Cash','Position','P1','P2','P3','P4','P5','P6','MTM']]   

    returns = SP['MTM'][StartDate-1:-1].pct_change()
    returns = returns[returns != 0]
    entry.append(thre)
    MTM.append(SP['MTM'][len(SP)-1])
    sharpe.append(np.sqrt(12) * returns.mean() / returns.std())   
    mdd.append(max_drawdown(SP['MTM']))
    


In [21]:
d= {'Pullback': entry, 'MTM':MTM, 'Sharpe': sharpe, 'Max drawndown':mdd}
a = pd.DataFrame(data = d)
a = a[['Pullback','MTM','Sharpe','Max drawndown']]
a

,Pullback,MTM,Sharpe,Max drawndown
0,0.01,45599198.016060,0.521591,0.525559
1,0.02,45599198.016060,0.521591,0.525559
2,0.03,45599198.016060,0.521591,0.525559
3,0.04,42713938.081550,0.518620,0.525559
4,0.05,42713938.081550,0.518620,0.525559
5,0.06,42713938.081550,0.518620,0.525559
6,0.07,10233393.142540,0.448349,0.586513
7,0.08,6567622.006633,0.388832,0.596613
8,0.09,7114753.350094,0.412709,0.596613
9,0.10,8299862.880559,0.446968,0.596613


In [22]:
a.to_csv('test results.csv')

In [ ]:
plot(SP['Date'],SP['MTM'])

In [ ]:
for x in [float(j)/100 for j in range(1,6,1)]:
    print (x)

In [ ]:
SP

In [ ]:
x = SP['Pullback'][59:].values

plt.hist(x, bins=np.histogram(x)[1].tolist())
plt.show()

In [ ]:
np.histogram(x)

In [23]:
SP

,Date,AdjClose,Pullback,T Bill,MTM
0,1950-01-31,17.049999,NaN,0,0.000000
1,1950-02-28,17.219999,NaN,0,0.000000
2,1950-03-31,17.290001,NaN,0,0.000000
3,1950-04-30,17.959999,NaN,0,0.000000
4,1950-05-31,18.780001,NaN,0,0.000000
5,1950-06-30,17.690001,NaN,0,0.000000
6,1950-07-31,17.840000,NaN,0,0.000000
7,1950-08-31,18.420000,NaN,0,0.000000
8,1950-09-30,19.450001,NaN,0,0.000000
9,1950-10-31,19.530001,NaN,0,0.000000
